# Chapter 4. Classes & OOP
## 4.1 Object Comparisons: “is” vs “==”

In [77]:
import datetime
import traceback as tb
import copy
from abc import ABCMeta, abstractmethod
from collections import namedtuple
import json
import math

In [64]:
a = [1, 2, 3]
b = a

In [65]:
a

[1, 2, 3]

In [66]:
b

[1, 2, 3]

In [67]:
a == b

True

In [68]:
a is b

True

In [69]:
b is a

True

In [70]:
a = [4, 5, 6]
c = list(a)
c == a

True

In [71]:
c is a

False

## 4.2 String Conversion (Every Class Needs a \_\_repr\_\_)

In [72]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage

In [73]:
my_car = Car('red', 37281)
print(my_car)

In [74]:
my_car

In [75]:
print(my_car.color, my_car.mileage)

red 37281


In [76]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage
    
    def __str__(self):
        return f'a {self.color} car'

In [77]:
my_car = Car('red', 37281)
my_car

In [78]:
str(my_car)

'a red car'

In [79]:
print(my_car)

a red car


### Метод \_\_str\_\_ против \_\_repr\_\_

In [80]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage
    
    def __repr__(self):
        return '__repr__ for Car'
    
    def __str__(self):
        return '__str__ for Car'

In [81]:
my_car = Car('red', 37281)
my_car

__repr__ for Car

In [82]:
print(my_car)

__str__ for Car


In [83]:
f"{my_car}"

'__str__ for Car'

In [84]:
str([my_car])

'[__repr__ for Car]'

In [85]:
repr(my_car)

'__repr__ for Car'

In [86]:
today = datetime.date.today()
today

datetime.date(2020, 5, 25)

In [87]:
str(today)

'2020-05-25'

In [88]:
repr(today)

'datetime.date(2020, 5, 25)'

In [89]:
# Good:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage
    
    def __repr__(self):
        return f"Car({self.color!r}, {self.mileage!r})"

In [90]:
my_car = Car('red', 37281)
my_car

Car('red', 37281)

In [91]:
# Better:
class Another_car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage
    
    def __repr__(self):
        return f"{self.__class__.__name__}({self.color!r}, {self.mileage!r})"

In [92]:
my_car = Another_car('red', 37281)
my_car

Another_car('red', 37281)

In [93]:
print(my_car)

Another_car('red', 37281)


In [94]:
str(my_car)

"Another_car('red', 37281)"

## 4.3 Defining Your Own Exception Classes

In [95]:
class NameTooShortError(ValueError):
    pass

def validate(name):
    if len(name) < 10:
        raise NameTooShortError(name)

In [100]:
try:
    validate('joe')
except NameTooShortError as e:
    print(repr(e))
    tb.print_tb(e.__traceback__)

NameTooShortError('joe')


  File "<ipython-input-100-dcdbba636258>", line 2, in <module>
    validate('joe')
  File "<ipython-input-95-4ee2fd66279c>", line 6, in validate
    raise NameTooShortError(name)


In [101]:
class BaseValidationError(ValueError):
    pass

class NameTooShortError(BaseValidationError):
    pass

class NameTooLongError(BaseValidationError):
    pass

class NameTooCuteError(BaseValidationError):
    pass

In [102]:
try:
    validate('joe')
except BaseValidationError as e:
    print(repr(e))
    tb.print_tb(e.__traceback__)

NameTooShortError('joe')


  File "<ipython-input-102-c49b43d3ffc4>", line 2, in <module>
    validate('joe')
  File "<ipython-input-95-4ee2fd66279c>", line 6, in validate
    raise NameTooShortError(name)


## 4.4 Cloning Objects for Fun and Profit
### Making Shallow Copies

In [103]:
xs = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
ys = list(xs) # Make a shallow copy

In [104]:
xs.append(['new sublist'])
xs

[[1, 2, 3], [4, 5, 6], [7, 8, 9], ['new sublist']]

In [105]:
ys

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

In [106]:
xs[1][0] = 'X'
xs

[[1, 2, 3], ['X', 5, 6], [7, 8, 9], ['new sublist']]

In [107]:
ys

[[1, 2, 3], ['X', 5, 6], [7, 8, 9]]

### Making Deep Copies

In [109]:
xs = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
zs = copy.deepcopy(xs)

In [110]:
xs

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

In [111]:
zs

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

In [112]:
xs[1][0] = 'X'
xs

[[1, 2, 3], ['X', 5, 6], [7, 8, 9]]

In [113]:
zs

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

### Copying Arbitrary Objects

In [114]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f'Point({self.x!r}, {self.y!r})'

In [115]:
a = Point(23, 42)
b = copy.copy(a)

In [116]:
a

Point(23, 42)

In [117]:
b

Point(23, 42)

In [118]:
a is b

False

In [119]:
class Rectangle:
    def __init__(self, topleft, bottomright):
        self.topleft = topleft
        self.bottomright = bottomright
    
    def __repr__(self):
        return (f'Rectangle({self.topleft!r}, {self.bottomright!r})')

In [120]:
rect = Rectangle(Point(0, 1), Point(5, 6))
srect = copy.copy(rect)

In [121]:
rect

Rectangle(Point(0, 1), Point(5, 6))

In [122]:
srect

Rectangle(Point(0, 1), Point(5, 6))

In [123]:
rect is srect

False

In [124]:
rect.topleft.x = 999
rect

Rectangle(Point(999, 1), Point(5, 6))

In [125]:
srect

Rectangle(Point(999, 1), Point(5, 6))

In [126]:
drect = copy.deepcopy(srect)
drect.topleft.x = 222
rect

Rectangle(Point(999, 1), Point(5, 6))

In [127]:
drect

Rectangle(Point(222, 1), Point(5, 6))

## 4.5 Abstract Base Classes Keep Inheritance in Check

In [128]:
class Base:
    def foo(self):
        raise NotImplementedError()
    
    def bar(self):
        raise NotImplementedError()


class Concrete(Base):
    def foo(self):
        return 'foo() called'
    # Oh no, we forgot to override bar()...
    # def bar(self):
    # return "bar() called"

In [130]:
b = Base()
try:
    b.foo()
except NotImplementedError as e:
    print(repr(e))
    tb.print_tb(e.__traceback__)

NotImplementedError()


  File "<ipython-input-130-28093c331ea6>", line 3, in <module>
    b.foo()
  File "<ipython-input-128-8912173218fe>", line 3, in foo
    raise NotImplementedError()


In [131]:
c = Concrete()
c.foo()

'foo() called'

In [132]:
try:
    c.bar()
except NotImplementedError as e:
    print(repr(e))
    tb.print_tb(e.__traceback__)

NotImplementedError()


  File "<ipython-input-132-d15904cee8bf>", line 2, in <module>
    c.bar()
  File "<ipython-input-128-8912173218fe>", line 6, in bar
    raise NotImplementedError()


In [134]:
class Base(metaclass=ABCMeta):
    @abstractmethod
    def foo(self):
        pass
    
    @abstractmethod
    def bar(self):
        pass


class Concrete(Base):
    def foo(self):
        pass
    # We forget to declare bar() again...

In [135]:
assert issubclass(Concrete, Base)

In [139]:
try:
    c = Concrete()
except TypeError as e:
    print(repr(e))
    tb.print_tb(e.__traceback__)

TypeError("Can't instantiate abstract class Concrete with abstract methods bar")


  File "<ipython-input-139-2380b0a84b99>", line 2, in <module>
    c = Concrete()


## 4.6 What Namedtuples Are Good For
### Namedtuples to the Rescue

In [3]:
Car = namedtuple('Car' , 'color mileage')

In [10]:
my_car = Car('red', 3812.4)

In [12]:
my_car.color

'red'

In [14]:
my_car.mileage

3812.4

In [15]:
color, mileage = my_car
print(color, mileage)

red 3812.4


In [16]:
print(*my_car)

red 3812.4


In [17]:
my_car

Car(color='red', mileage=3812.4)

In [20]:
try:
    my_car.color = 'blue'
except AttributeError as e:
    print(repr(e))
    tb.print_tb(e.__traceback__)

AttributeError("can't set attribute")


  File "<ipython-input-20-8e06d69a967a>", line 2, in <module>
    my_car.color = 'blue'


### Subclassing Namedtuples

In [21]:
Car = namedtuple('Car', 'color mileage')

class MyCarWithMethods(Car):
    def hexcolor(self):
        if self.color == 'red':
            return '#ff0000'
        else:
            return '#000000'

In [22]:
c = MyCarWithMethods('red', 1234)
c.hexcolor()

'#ff0000'

In [24]:
Car = namedtuple('Car', 'color mileage')
ElectricCar = namedtuple('ElectricCar', Car._fields + ('charge',))

In [25]:
ElectricCar._fields

('color', 'mileage', 'charge')

In [26]:
ElectricCar('red', 1234, 45.0)

ElectricCar(color='red', mileage=1234, charge=45.0)

### Built-in Helper Methods

In [29]:
my_car._asdict()

{'color': 'red', 'mileage': 3812.4}

In [31]:
json.dumps(my_car._asdict())

'{"color": "red", "mileage": 3812.4}'

In [33]:
my_car._replace(color='blue')

Car(color='blue', mileage=3812.4)

In [34]:
Car._make(['red', 999])

Car(color='red', mileage=999)

## 4.7 Class vs Instance Variable Pitfalls

In [38]:
class Dog:
    num_legs = 4 # <- Class variable
    
    def __init__(self, name):
        self.name = name # <- Instance variable

In [39]:
jack = Dog('Jack')
jill = Dog('Jill')
jack.name, jill.name

('Jack', 'Jill')

In [40]:
jack.num_legs, jill.num_legs

(4, 4)

In [41]:
Dog.num_legs

4

In [42]:
try:
    Dog.name
except AttributeError as e:
    print(repr(e))
    tb.print_tb(e.__traceback__)

AttributeError("type object 'Dog' has no attribute 'name'")


  File "<ipython-input-42-60b7527a95c3>", line 2, in <module>
    Dog.name


In [46]:
Dog.num_legs = 6
jack.num_legs, jill.num_legs

(6, 6)

In [47]:
Dog.num_legs = 4
jack.num_legs = 6

In [48]:
jack.num_legs, jill.num_legs, Dog.num_legs

(6, 4, 4)

In [49]:
jack.num_legs, jack.__class__.num_legs

(6, 4)

### A Dog-free Example

In [50]:
class CountedObject:
    num_instances = 0
    
    def __init__(self):
        self.__class__.num_instances += 1

In [51]:
CountedObject.num_instances

0

In [52]:
CountedObject().num_instances

1

In [53]:
CountedObject().num_instances

2

In [54]:
CountedObject().num_instances

3

In [55]:
CountedObject.num_instances

3

In [56]:
# WARNING: This implementation contains a bug
class BuggyCountedObject:
    num_instances = 0
    
    def __init__(self):
        self.num_instances += 1 # !!!

In [57]:
BuggyCountedObject.num_instances

0

In [58]:
BuggyCountedObject().num_instances

1

In [59]:
BuggyCountedObject().num_instances

1

In [61]:
BuggyCountedObject.num_instances

0

## 4.8 Instance, Class, and Static Methods Demystified

In [62]:
class MyClass:
    def method(self):
        return 'instance method called', self
    
    @classmethod
    def classmethod(cls):
        return 'class method called', cls
    
    @staticmethod
    def staticmethod():
        return 'static method called'

### Let’s See Them in Action!

In [63]:
obj = MyClass()
obj.method()

('instance method called', <__main__.MyClass at 0x20ed4971a00>)

In [64]:
MyClass.method(obj)

('instance method called', <__main__.MyClass at 0x20ed4971a00>)

In [65]:
obj.classmethod()

('class method called', __main__.MyClass)

In [66]:
obj.staticmethod()

'static method called'

In [67]:
MyClass.classmethod()

('class method called', __main__.MyClass)

In [68]:
MyClass.staticmethod()

'static method called'

In [69]:
try:
    MyClass.method()
except TypeError as e:
    print(repr(e))
    tb.print_tb(e.__traceback__)

TypeError("method() missing 1 required positional argument: 'self'")


  File "<ipython-input-69-bd6dec52231a>", line 2, in <module>
    MyClass.method()


In [70]:
class Pizza:
    def __init__(self, ingredients):
        self.ingredients = ingredients
    
    def __repr__(self):
        return f'Pizza({self.ingredients!r})'

In [71]:
Pizza(['cheese', 'tomatoes'])

Pizza(['cheese', 'tomatoes'])

### Delicious Pizza Factories With @classmethod

In [72]:
class Pizza:
    def __init__(self, ingredients):
        self.ingredients = ingredients
    
    def __repr__(self):
        return f'Pizza({self.ingredients!r})'
    
    @classmethod
    def margherita(cls):
        return cls(['mozzarella', 'tomatoes'])
    
    @classmethod
    def prosciutto(cls):
        return cls(['mozzarella', 'tomatoes', 'ham'])

In [73]:
pizza_margherita = Pizza.margherita()

In [74]:
pizza_margherita

Pizza(['mozzarella', 'tomatoes'])

In [75]:
pizza_prosciutto = Pizza.prosciutto()

In [76]:
pizza_prosciutto

Pizza(['mozzarella', 'tomatoes', 'ham'])

### When To Use Static Methods

In [78]:
class Pizza:
    def __init__(self, radius, ingredients):
        self.radius = radius
        self.ingredients = ingredients
    
    def __repr__(self):
        return (f'Pizza({self.radius!r}, {self.ingredients!r})')
    
    def area(self):
        return self.circle_area(self.radius)
    
    @staticmethod
    def circle_area(r):
        return r ** 2 * math.pi

In [79]:
p = Pizza(4, ['mozzarella', 'tomatoes'])
p

Pizza(4, ['mozzarella', 'tomatoes'])

In [81]:
p.area()

50.26548245743669

In [82]:
 Pizza.circle_area(4)

50.26548245743669